In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
regression_energy=pd.read_csv('/content/energydata_complete.csv')

In [7]:
regression_energy.head()


,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [54]:
column_names=('Appliances','T1','RH_1','T2','RH_2','T3',
	'RH_3','T4','RH_4','T5','RH_5','T6','RH_6','T7','RH_7', 'T8','RH_8',',T9','RH_9','T_out',
'Press_mm_hg','RH_out','Windspeed','Visibility','Tdewpoint',	'rv1'	,'rv2')

In [55]:
energy_data=regression_energy.drop(columns=['date','lights'])

**Normalize Energy applinaces dataset**

In [57]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
normalized_df=pd.DataFrame(scaler.fit_transform(energy_data),columns=column_names)
prediction=normalized_df.drop(columns=['Appliances'])
target_var=normalized_df['Appliances']

In [66]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(prediction,target_var,
                                               test_size=0.3, random_state=42)

In [67]:
from sklearn.linear_model import LinearRegression
linear_model=LinearRegression()

In [68]:
linear_model.fit(x_train,y_train)

LinearRegression()

In [69]:
predicted_values=linear_model.predict(x_test)

**Mean Absolute Error (MAE)**

In [70]:
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test,predicted_values)
round(mae,3)

0.05

**Residual Sum of Squares**

In [71]:
import numpy as np
rss=np.sum(np.square(y_test-predicted_values))
round(rss,2)


45.35

**Root Mean Squared Error**

In [73]:
from sklearn.metrics import mean_squared_error
rmse=np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

**Coefficient of determination or R-squared**

In [74]:
from sklearn.metrics import r2_score
r2_score=r2_score(y_test,predicted_values)
round(r2_score,2)

0.15

**Ridge Regression**

In [78]:
from sklearn.linear_model import Ridge
ridge_reg=Ridge(alpha=0.4)
ridge_reg.fit(x_train,y_train)


Ridge(alpha=0.4)

In [84]:
from sklearn.linear_model import Lasso
lasso_reg=Lasso(alpha=0.001)
lasso_reg.fit(x_train,y_train)

Lasso(alpha=0.001)

In [85]:
from sklearn.metrics import mean_squared_error
rmse=np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

In [81]:
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [82]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

In [83]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.411071,-0.000000
1,T_out,-0.321860,-0.262172,0.000000
2,T2,-0.236178,-0.201397,0.000000
3,",T9",-0.189941,-0.188916,-0.000000
4,RH_8,-0.157595,-0.156830,-0.000110
5,RH_out,-0.077671,-0.054724,-0.049557
6,RH_7,-0.044614,-0.045977,-0.000000
7,RH_9,-0.039800,-0.041367,-0.000000
8,T5,-0.015657,-0.019853,-0.000000
9,T1,-0.003281,-0.018406,0.000000


**from the result, RH2 is the lowest and RH1 is the highest**